# Кластеризация

В данной тетради рассматривается решение задачи кластерного анализа алгоритмами иерархической кластеризации. Показаны возможности библиотек Python непосредственно по решению задачи кластеризации, извлечению нужной структуры кластеров и их графического представления.    
Общая схема решения задачи:     
1. Чтение исходных данных (формат файла, формат записей, кодировка и т.п.);
2. Получение предварительной информации об имеющимся массива (просмотр заголовков, размерность, просмотр части данных, получение информации о типах данных, преобразование типов, поиск пропусков, заполнение пропусков);
3. Аккуратные данные (поиск и заполнение пропусков, поиск и удаление дубликатов, формирование итогового набора данных для кластеризации);
4. Иерархическая кластеризация: агломеративный алгоритм.


Дополнительные источники для изучения указаны в конце настоящей тетради. 

## 1. Чтение исходных данных

Для ознакомления с необходимым материал рекомендую книгу [1, глава 6, стр. 175].     
В рассматриваемом примере исходные данные хранятся в txt-файле, разделенные знаком табуляции.    
Для чтения воспользуемся функцией `.read_table()`.    
В качестве параметров передадим
- `path` -- путь к файлу;
- `encoding` -- исходную кодировку, в которой хранятся данные;
- `sep` -- последовательность символов или регулярное выражение, служащее для разделения полей в строке: ',',';','\t'(знак табуляции), '\s+' -- если поля разделены переменным числом пробелов;
- `nrows` -- количество читаемых строк от начала файла.

Подключаем библиотеку `pandas` и загружаем исходные данные в датафрейм `df`. 

ВАЖНО: В данном случае с целью упрощения и повышения производительности прочитаны только первых 100 строк исходного массива.

In [16]:
import pandas as pd
pd.set_option('display.precision',3)
df=pd.read_table('mobile.txt', encoding='1251', sep='\t', nrows=100)

## 2. Получение предварительной информации об имеющимся массива   

Для получения предварительной информации об анализируемом массиве можно воспользоваться следующими возможностями Python    
- Просмотр с помощью свойства `.head()` указанного числа первых строк датафрейма. По умолчанию выводится 5 первых строк датафрейма;
- Размерность датафрейма можно узнать с помощью свойства .shape;
- Просмотр заголовков столбов с помощью свойства .columns;
- Просмотр информации о типах переменных с помощью свойства `.info()`

In [17]:
df.head()

,Код,Возраст,Среднемесячный расход,Средняя продолжительность разговоров,Звонков днем за месяц,Звонков вечером за месяц,Звонков ночью за месяц,Звонки в другие города,Звонки в другие страны,Доля звонков на стационарные телефоны,Количество SMS за месяц
0,0,24,121.54,2.4,12,65,5,0,0,5,56
1,1,51,287.51,1.7,111,109,1,44,0,6,1
2,2,41,113.70,2.1,41,27,0,0,0,1,36
3,3,35,410.23,5.6,47,49,0,0,0,11,23
4,4,26,537.60,4.8,58,77,4,0,0,16,29


In [ ]:
df.iloc[:,1:6:2]

ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ: С помощью свойства `.tail()` можно просмотреть заданное число последних строк массива. По умолчанию выводится 5 строк 

+ Размерность датафрейма можно узнать с помощью свойства `.shape`    
ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ: количество строк датафрейма можно узнать с помощью метода `len()`

In [ ]:
df.shape

+ Просмотр заголовков столбов с помощью свойства `.columns` 

In [ ]:
df.columns

## 3. Аккуратные данные

Аккуратные данные --- данные, пригодные для дальнейшего анализа. Для получения аккуратных данных необходимо разрешить следующие проблемы, связанные с имеющимися данными (список может быть расширен):     
- имена объектов или признаков отличаются от желаемых (требуемых);  
- есть пропущенные данные;
- значения указаны не в тех единицах измерения, которые требуются;
- временной период выборки наблюдений не тот;
- переменные являются категориальными, а требуются количественные;
- присутсвует шум в данных;
- информация неверного типа;
- данные неправильно ориентированы по осям;
- данные неправильно нормализованы;
- данные дублизуются.

## 3.1 Поиск и импутация пропущенных наблюдений
- Вывод общей информации о наличие пропусков во всех переменных можно осуществить, воспользовавшись цепочкой методов `.isnull()` и `.sum()`

In [ ]:
df.isnull().sum()

## 3.2 Типы признаков

Для просмотра информации о типах наблюдаемых признаков у объектов воспользуемся свойством `.info()`

In [ ]:
df.info()

## 3.3 Генерирование входного массива и нормализация признаков 

In [ ]:
X=df.iloc[:,:].values
X=(X-X.mean(axis=0))/X.std(axis=0)
#print(X.mean(axis=0)) #просмотр массива выборочных средних -- они должны быть равны 0 
#print(X.std(axis=0)) # просмотр массива стандартных отклонений --- они должны быть равными 1
#print(X) # просмотр входного массива --- аккуратные данные!!!!

## 4. Иерархическая кластеризация: агломеративный алгоритм

### 4.1 Реализация разбинения, матрица связей и ее интерпретация, визуализация

Для дополнительного изучения материала рекомендуется воспользоваться ресурсами [2, 3]    
Иерархическая кластеризация реализована в модуле `scipy.cluster.hierarchy`.    
Импортируем из этого модуля методы:    
- `.linkage()` -- выполняет иерархическую (агломеративную) кластеризацию; 
- `.dendrogram()` -- строит дендрограмму;
- `.fcluster()` -- 

Метод `.linkage()` имеет следующую спецификацию `linkage(X[, method, metric, optimal_ordering])`:    
- X -- матрица попарных расстояний или исходных данных (в матрице не должно быть пробелов или категориальных значений)
- method -- правило, по которому будут рассчитываться расстояния между кластерами:    
    - `single`
    - `complete`
    - `average`
    - `weighted`
    - `centroid`
    - `median`
    - `ward`
- metric -- метрика: `braycurtis`, `canberra`, `chebyshev`, `cityblock`, `correlation`, `cosine`, `dice`, `euclidean`, `hamming`, `jaccard`, `jensenshannon`, `kulsinski`, `mahalanobis`, `matching`, `minkowsk`, `rogerstanimoto`, `russellrao`, `seuclidean`, `sokalmichener`, `sokalsneath`, `sqeuclidean`, `yule`.

Результатом выполнения метода `.linkage()` является массив связей кластера. Обозначим его `Z`. Ниже дадим интерпретацию элементам массива `Z`.   

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
Z = linkage(X, method='ward') 
plt.title('Иерархическая кластеризация: дендрограмма')
plt.xlabel('Объеты/ кластеры')
plt.ylabel('Расстояние')
dendrogram(Z) #рекомендуется посмотреть и другие параметры этого метода 
plt.show() #отобразить дендрограмму 
r.savefig('1.png', dpi = 300) #сохранить рисунок

In [ ]:
print(Z)

Дадим интерпретацию элементам массив связей кластера `Z`. В первом и втором столбцах массива `Z` указаны индексы кластеров (в том числе синглетонов), объединяющих на текущей итерации. В третьем столбце находится расстояние между кластерами. В четвертом столбце индекс нового кластера, показыващий число синглетонов в новом кластере. Например, строка `Z[0]` содержит следующее `array([47., 50., 0.4064717, 2.])`. Элементы этой строки показывают, что на первой (напомним, что индексация массивов начинается с 0) итерации алгоритма объединяются кластеры (здесь синглетоны -- кластер, состоящий из одного элемента) с номерами `47` и `50`, расстояние между кластерами равно `0.4064717`, новому кластеру присвоен номер `2` -- по числу синглетонов в образовавшемся кластере, и т.д.    
Напомним, что в исходном массиве число объектов 100, проиндексированных от 0 до 99. Обратим внимание на строку массива `Z` с индексом 3. Строка `Z[3]` имеет вид `array([100., 101., 0.60829392, 4.])`. Имеем, что на данной итерации объединяются кластеры с индексами `100` и `101`. Однако в исходном множестве объектов максимальный индекс равен `99`. Индексы такие, что $idx \geqslant len(X)$ соответствуют кластерам, которые объединились ранее и находятся в массиве связей кластера в строке `Z[idx-len(X)]`, где $len(X)=100$. Имеем, индекс 100 соответствует кластеру, который образовался на 1 итерации алгоритма и находится в строке `Z[0]`. Видим, что в этой строке объединяются синглетоны с индексами `47` и `50`. Аналогичным образом, индекс 101 соответствует строке массива `Z[1]` --- здесь объединились синглетоны с индексами `46` и `51`. В результате в образовавшемся кластере находятся `4` синглетона.

### 4.2 Анализ результатов кластеризации: усеченная дендрограмма, выбор кластеров, вывод состава кластеров

Если количество элементов, подлежащих кластеризации велико, то можно отобразить усеченную дендрограмму. Усечение используется для сжатия дендрограммы. Для того, чтобы построить усеченную дендрограмму необходимо в команде `.dendrogram()` использовать режим `truncate_mode` со значением `lastp` или `level`.     
Если `truncate_mode='lastp'`, то отображается `p` последних образовавшихся кластера. В матрице связей $Z$ это соответствует строкам $Z[n-p-2:end]$. По оси абцисс в скобках указывается число объектов, содержащихся в кластерах.

In [ ]:
dendrogram(Z, 
           truncate_mode = 'lastp', #pвключен режим показывать последние 'p' кластеров  
           p=11,                     #отобразить последние 11 кластеров 
           leaf_rotation = 0,       #угол поворота надписей по оси  
           leaf_font_size = 20,     #размер шрифта  
           show_contracted = True)  # рисовать черные точки на высотах предыдущих слияний кластеров
plt.show()

Для выделения некоторого разбиения объектов на кластеры воспользуемся функцией `fcluster()`, которая на основе построенной матрицы связей `Z` выделяет некоторое разбиение на кластеры, исходя из задаваемых польователем критериев. В качестве критерия можно указать либо максимальное количество кластеров `criterion='maxclust'`, либо расстоние `criterion='distance'`.     
 
В первом случае мы указываем требуемое (желаемое) максимальное число кластеров, т.е. используем критерий `criterion='maxclust'`. На выходе имеем массив с указанием номера кластера, которому принадлежит соответствующий исходный объект 

In [ ]:
label=fcluster(Z, 7, criterion='maxclust')
print(label)

Для анализа каждого из выделенных кластеров реализуем следующие действия:    
1) добавим в исходный датафрейм новый столбец `Номер кластера`;    
2) в цикле `for` с использоанием механизма группировки датафрейма `groupby` выведем объекты кластеров

In [ ]:
df.loc[:,'Номер кластера']=label # добавление нового столбца с метками 
for i, group in df.groupby('Номер кластера'): # вывод элементов каждого кластера  
    print('=' * 10)
    print('cluster {}'.format(i))
    print(group)

Во втором случае мы указываем расстояние по оси ординат, на котором будем проводить линию среза, т.е. используем критерий `criterion='distance'`. На выходе имеем массив с указанием номера кластера, которому принадлежит соответствующий исходный объект.     
Пусть, например, мы хотим узнать какие кластеры объединились на расстоянии 7. Для наглядности на дендрограмме нарисуем соответствующую линию среза `y=10`.   

In [ ]:
plt.axhline(y=10)
dendrogram(Z, 
           truncate_mode = 'lastp', #pвключен режим показывать последние 'p' кластеров  
           p=11,                     #отобразить последние 11 кластеров 
           leaf_rotation = 0,       #угол поворота надписей по оси  
           leaf_font_size = 20,     #размер шрифта  
           show_contracted = True)  # рисовать черные точки на высотах предыдущих слияний кластеров
plt.show()

На рисунке показано, что на этом уровне имеется 5 кластеров. Выделим эти кластеры с использованием функции `fcluster()`, задав критерий расстояния. Имеем

In [ ]:
label1=fcluster(Z, 10, criterion='distance')
print(label1)

Видим, что построено распределение исходных элементов на 5 кластеров. Для анализа каждого кластера можно проделать последовательность действий, аналогичную выше рассмотренной.  

Литература
1. Маккинли У. Python и анализ данных. - М.: ДМК Пресс, 2015. – 482 с.
2. https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/
3. https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html

# Замечания 
Отредактировать текст с научной предметной лексикой -- линия среза 

# Черновики

In [ ]:
Z = linkage(X, method='ward') 
r=plt.figure()
dendrogram(Z, 
#           truncate_mode = 'lastp', #включен режим показывать последние 'p' кластеров  
#           p=4,                     #задается параметр 'p' 
           leaf_rotation = 0,       #угол поворота надписей по оси  
           leaf_font_size = 20,     #размер шрифта  
           show_contracted = True)
r.savefig('2.png', dpi = 300)

In [ ]:
label=fcluster(Z, 2, criterion='distance')

In [ ]:
X.loc[:,'label']=label

In [ ]:
for i, group 

## 5. Неиерархическая кластеризация: метод k-средних

In [15]:
from scipy.cluster.vq import vq, kmeans, whiten, kmeans2

In [ ]:
whitened = whiten(X)
codebook, distortion=kmeans(whitened,3)

In [ ]:
centroid, label = kmeans2(X, 3, minit='points')

In [ ]:
label